In [ ]:
# 🅿️ Dynamic Pricing for Urban Parking Lots
**Capstone Project – Summer Analytics 2025**  
Hosted by **Consulting & Analytics Club × Pathway**

---

## 📌 Overview

Urban parking spaces are often inefficiently utilized due to static pricing. This project implements a real-time dynamic pricing system for 14 parking lots using ML models and real-time streaming pipelines.

💡 This pricing engine dynamically updates prices based on:
- Occupancy & queue length
- Traffic congestion
- Special events
- Vehicle type
- Competitor pricing


SyntaxError: invalid character '×' (U+00D7) (1214167536.py, line 3)

In [7]:
import pandas as pd
import numpy as np

# Read the CSV file (make sure you upload it to the 'Files' section in Colab)
df = pd.read_csv('data/dataset.csv')  # Adjust path if needed
df.head()



,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,4/10/2016,7:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,4/10/2016,8:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,4/10/2016,8:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,4/10/2016,9:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,4/10/2016,9:59:00


In [10]:
def baseline_linear_pricing(df, alpha=2, base_price=10, min_price=5, max_price=20):
    df = df.copy()

    # Combine date and time columns, auto-parse format with dayfirst=True
    df['Timestamp'] = pd.to_datetime(
        df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
        dayfirst=True, errors='coerce'
    )

    # Drop rows with invalid timestamps (optional safety check)
    df = df.dropna(subset=['Timestamp'])

    # Sort data
    df = df.sort_values(by=['SystemCodeNumber', 'Timestamp'])

    df['Price_Model1'] = 0.0

    for lot_id, group in df.groupby('SystemCodeNumber'):
        prices = [base_price]
        for i in range(1, len(group)):
            occ = group.iloc[i]['Occupancy']
            cap = group.iloc[i]['Capacity']
            price = prices[-1] + alpha * (occ / cap)
            price = min(max(price, min_price), max_price)
            prices.append(price)
        df.loc[group.index, 'Price_Model1'] = prices

    return df

# Call the function
df = baseline_linear_pricing(df)
df[['SystemCodeNumber', 'Timestamp', 'Price_Model1']].head()


,SystemCodeNumber,Timestamp,Price_Model1
0,BHMBCCMKT01,2016-10-04 07:59:00,10.000000
1,BHMBCCMKT01,2016-10-04 08:25:00,10.221837
2,BHMBCCMKT01,2016-10-04 08:59:00,10.499133
3,BHMBCCMKT01,2016-10-04 09:32:00,10.870017
4,BHMBCCMKT01,2016-10-04 09:59:00,11.389948


In [11]:
def demand_based_pricing(df, base_price=10):
    traffic_map = {'low': 1, 'medium': 2, 'high': 3}
    vehicle_map = {'bike': 0.5, 'car': 1.0, 'truck': 1.5}

    df = df.copy()
    df['TrafficWeight'] = df['TrafficConditionNearby'].map(traffic_map)
    df['VehicleWeight'] = df['VehicleType'].map(vehicle_map)

    # Coefficients
    a, b, g, d, e, l = 0.4, 0.2, 0.3, 0.5, 0.2, 0.5

    df['DemandRaw'] = (
        a * (df['Occupancy'] / df['Capacity']) +
        b * df['QueueLength'] -
        g * df['TrafficWeight'] +
        d * df['IsSpecialDay'] +
        e * df['VehicleWeight']
    )

    df['DemandNorm'] = (df['DemandRaw'] - df['DemandRaw'].min()) / (df['DemandRaw'].max() - df['DemandRaw'].min())
    df['Price_Model2'] = base_price * (1 + l * df['DemandNorm'])
    df['Price_Model2'] = df['Price_Model2'].clip(lower=5, upper=20)

    return df

df = demand_based_pricing(df)
df[['SystemCodeNumber', 'Timestamp', 'Price_Model2']].head()


,SystemCodeNumber,Timestamp,Price_Model2
0,BHMBCCMKT01,2016-10-04 07:59:00,10.499798
1,BHMBCCMKT01,2016-10-04 08:25:00,10.502904
2,BHMBCCMKT01,2016-10-04 08:59:00,10.818157
3,BHMBCCMKT01,2016-10-04 09:32:00,10.846110
4,BHMBCCMKT01,2016-10-04 09:59:00,10.741284


In [12]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # meters
    phi1, phi2 = radians(lat1), radians(lat2)
    dphi = radians(lat2 - lat1)
    dlambda = radians(lon2 - lon1)
    a = sin(dphi / 2)**2 + cos(phi1) * cos(phi2) * sin(dlambda / 2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))

def competitive_pricing(df):
    df = df.copy()
    df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'])
    lot_coords = df.groupby('SystemCodeNumber')[['Latitude', 'Longitude']].first().to_dict('index')

    nearby_lots = {}
    for a, coord_a in lot_coords.items():
        nearby_lots[a] = []
        for b, coord_b in lot_coords.items():
            if a != b and haversine(coord_a['Latitude'], coord_a['Longitude'], coord_b['Latitude'], coord_b['Longitude']) <= 500:
                nearby_lots[a].append(b)

    df['Price_Model3'] = df['Price_Model2']
    for idx, row in df.iterrows():
        lot = row['SystemCodeNumber']
        timestamp = row['Timestamp']
        own_price = row['Price_Model2']
        occupancy_ratio = row['Occupancy'] / row['Capacity']
        competitors = nearby_lots.get(lot, [])
        comp_prices = df[
            (df['SystemCodeNumber'].isin(competitors)) & (df['Timestamp'] == timestamp)
        ]['Price_Model2']

        if not comp_prices.empty:
            avg_comp_price = comp_prices.mean()
            if occupancy_ratio > 0.95 and own_price > avg_comp_price:
                new_price = own_price * 0.9
            elif occupancy_ratio < 0.7 and own_price < avg_comp_price:
                new_price = own_price * 1.1
            else:
                new_price = own_price
            df.at[idx, 'Price_Model3'] = min(max(new_price, 5), 20)
    return df

df = competitive_pricing(df)
df[['SystemCodeNumber', 'Timestamp', 'Price_Model3']].head()


,SystemCodeNumber,Timestamp,Price_Model3
0,BHMBCCMKT01,2016-04-10 07:59:00,11.549777
1,BHMBCCMKT01,2016-04-10 08:25:00,10.502904
2,BHMBCCMKT01,2016-04-10 08:59:00,11.899972
3,BHMBCCMKT01,2016-04-10 09:32:00,10.846110
4,BHMBCCMKT01,2016-04-10 09:59:00,11.815413


In [14]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
output_notebook()

def plot_prices(df, lot_id):
    df_lot = df[df['SystemCodeNumber'] == lot_id].copy().sort_values('Timestamp')
    source = ColumnDataSource(df_lot)

    p = figure(x_axis_type='datetime', title=f"Pricing for Lot {lot_id}", width=900, height=400)
    p.line(x='Timestamp', y='Price_Model1', source=source, color='blue', legend_label="Model 1 (Baseline)")

    # Only plot Model 2 and 3 if they exist in the DataFrame
    if 'Price_Model2' in df.columns:
        p.line(x='Timestamp', y='Price_Model2', source=source, color='green', legend_label="Model 2 (Demand)")
    if 'Price_Model3' in df.columns:
        p.line(x='Timestamp', y='Price_Model3', source=source, color='red', legend_label="Model 3 (Competitive)")

    p.legend.location = "top_left"
    p.xaxis.axis_label = 'Timestamp'
    p.yaxis.axis_label = 'Price ($)'
    show(p)

# Plot for the first unique SystemCodeNumber
plot_prices(df, lot_id=df['SystemCodeNumber'].unique()[0])


Loading BokehJS ...

In [ ]:
##  Conclusion

This project simulates real-time dynamic pricing for urban parking lots. We implemented three models:

- **Model 1**: Simple occupancy-based pricing
- **Model 2**: Rich demand-based pricing using traffic, queue, and vehicle type
- **Model 3**: Smart pricing with location-based competitive intelligence

All models were visualized using Bokeh for real-time interpretability.




SyntaxError: invalid character '📌' (U+1F4CC) (2654214454.py, line 13)